In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('train.csv')
df.info()
df.isna().sum()
# df.duplicated().sum()
# df.drop_duplicates()
label =  LabelEncoder()
df['model'] = label.fit_transform(df['model'])
df['motor_type'] = label.fit_transform(df['motor_type'])
# df['wheel'] = df['wheel'].map({'left': 1})
df['wheel'] = label.fit_transform(df['wheel'])
df['color'] = label.fit_transform(df['color'])
df['status'] = label.fit_transform(df['status'])
df['type'] = label.fit_transform(df['type'])
def convert_running(value):
    if 'km' in value:
        kilometers = int(value.split()[0])  
        miles = kilometers * 0.621371        
        return f"{miles:.2f} miles"          
    elif 'miles' in value:
        return value  
    else:
        return value 
df['running'] = df['running'].apply(convert_running)
df['running'] = df['running'].str.replace('miles', '')


df.info()

df['running'] = df['running'].str.strip()
df['running'] = pd.to_numeric(df['running'], errors='coerce')
df['running'] = df['running'].fillna(0).astype('int64')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1642 entries, 0 to 1641
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         1642 non-null   object 
 1   year          1642 non-null   int64  
 2   motor_type    1642 non-null   object 
 3   running       1642 non-null   object 
 4   wheel         1642 non-null   object 
 5   color         1642 non-null   object 
 6   type          1642 non-null   object 
 7   status        1642 non-null   object 
 8   motor_volume  1642 non-null   float64
 9   price         1642 non-null   int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 128.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1642 entries, 0 to 1641
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         1642 non-null   int32  
 1   year          1642 non-null   int64  
 2   motor_type    1642 non-null  

In [3]:
df.corr()['price']

model           0.173485
year            0.638699
motor_type      0.238964
running        -0.513134
wheel                NaN
color          -0.036846
type            0.037789
status         -0.282471
motor_volume    0.000885
price           1.000000
Name: price, dtype: float64

In [5]:
x_train = df.drop(['price'], axis = 1)
y_train = df['price']
x_train.shape, y_train.shape

((1642, 9), (1642,))

In [7]:
from sklearn.preprocessing import StandardScaler
scalar =  StandardScaler()
feature_standardise = scalar.fit_transform(x_train)
feature_standardise

array([[ 1.52639053,  1.09243073,  0.26699337, ..., -0.03413756,
        -0.45353122, -0.1384163 ],
       [ 0.05769246, -0.12234706,  0.26699337, ..., -0.03413756,
        -0.45353122, -0.1384163 ],
       [-0.67665657,  0.48504183,  0.26699337, ..., -0.03413756,
        -0.45353122, -0.1384163 ],
       ...,
       [ 0.79204149,  0.48504183,  0.26699337, ...,  1.08693988,
         0.97583205, -0.1384163 ],
       [ 0.79204149,  0.63688906,  0.26699337, ...,  1.08693988,
        -0.45353122, -0.1384163 ],
       [ 1.52639053,  1.09243073,  0.26699337, ..., -0.03413756,
        -0.45353122, -0.1384163 ]])

In [9]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np

# Updated parameter grid
param_distributions = {
    'n_estimators': [100, 200, 300, 400],  # Start with a larger base
    'learning_rate': [0.01, 0.05, 0.1, 0.2],  # More granular learning rates
    'max_depth': [3, 4, 5, 6, 7],  # Reduced depth range
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 3, 4],  # Reduced to avoid overfitting
    'subsample': [0.5, 0.75, 1.0, 1.25],
    'max_features': ['sqrt', 'log2', None],  # Focus on a few options
    'loss': ['ls', 'huber',  'quantile'],  # Remove 'lad' to reduce complexity
    'alpha': [0.75, 0.9, 1.5]  # Only used if loss='huber'
}

# Initialize the GradientBoostingRegressor with early stopping
gbr = GradientBoostingRegressor(random_state=42, n_iter_no_change=10, tol=0.0001)

# Perform RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=gbr,
    param_distributions=param_distributions,
    n_iter=100,
    scoring='neg_mean_absolute_error',
    cv=5,
    verbose=1,
    random_state=42,
    n_jobs=-1,
    return_train_score=True  # Optional: track training score
)

# Fit the model on standardized features
random_search.fit(feature_standardise, y_train)

# Get the best parameters
best_params = random_search.best_params_
print(f'Best parameters: {best_params}')

# Get the best model
best_model = random_search.best_estimator_
print(best_model)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


C:\Users\admin\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
330 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
165 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\admin\anaconda3\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "C:\Users\admin\anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\admin\anaconda3\Lib\site-packag

Best parameters: {'subsample': 0.75, 'n_estimators': 300, 'min_samples_split': 15, 'min_samples_leaf': 4, 'max_features': None, 'max_depth': 6, 'loss': 'huber', 'learning_rate': 0.05, 'alpha': 0.9}
GradientBoostingRegressor(learning_rate=0.05, loss='huber', max_depth=6,
                          min_samples_leaf=4, min_samples_split=15,
                          n_estimators=300, n_iter_no_change=10,
                          random_state=42, subsample=0.75)


In [11]:
# Predictions and evaluation
y_pred = best_model.predict(feature_standardise)
mae = mean_absolute_error(y_train, y_pred)
print(f'Mean Absolute Error: {mae:.2f}')


Mean Absolute Error: 1573.91


In [13]:
gbr = GradientBoostingRegressor(learning_rate=0.05, loss='huber', max_depth=6,
                          min_samples_leaf=4, min_samples_split=15,
                          n_estimators=300, n_iter_no_change=10,
                          random_state=42, subsample=0.75)
gbr.fit(feature_standardise, y_train)

GradientBoostingRegressor(learning_rate=0.05, loss='huber', max_depth=6,
                          min_samples_leaf=4, min_samples_split=15,
                          n_estimators=300, n_iter_no_change=10,
                          random_state=42, subsample=0.75)

In [15]:
from sklearn.metrics import  mean_squared_error, mean_absolute_error, r2_score

y_pred = gbr.predict(feature_standardise)
mse = mean_squared_error(y_train, y_pred)
print(f'Mean Squared Error with Gradient Boosting: {mse:.2f}')
mae = mean_absolute_error(y_train, y_pred)
print(mae)
r2 = r2_score(y_train, y_pred)
print(r2)


# Mean Squared Error with Gradient Boosting: 8957325.64
# 1573.9081696891762
# 0.8259524842024936

Mean Squared Error with Gradient Boosting: 8957325.64
1573.9081696891762
0.8259524842024936


In [17]:
test = pd.read_csv('test.csv')
from sklearn.preprocessing import LabelEncoder
label =  LabelEncoder()


test['model'] = label.fit_transform(test['model'])
test['motor_type'] = label.fit_transform(test['motor_type'])
test['wheel'] = label.fit_transform(test['wheel'])
# test['wheel'] = test['wheel'].map({'left': 1})
test['color'] = label.fit_transform(test['color'])
test['status'] = label.fit_transform(test['status'])
test['type'] = label.fit_transform(test['type'])
def convert_runnings(value):
    if 'km' in value:
        kilometers = int(value.split()[0])  
        miles = kilometers * 0.621371        
        return f"{miles:.2f} miles"          
    elif 'miles' in value:
        return value  
    else:
        return value 

test['running'] = test['running'].apply(convert_runnings)
test['running'] = test['running'].str.replace('miles', '')
x_test = test

In [19]:
x_test = x_test.drop(['Id'], axis = 1)

In [21]:
from sklearn.preprocessing import StandardScaler
scalar =  StandardScaler()
feature_standard = scalar.fit_transform(x_test)
feature_standard

array([[-0.62247891,  0.77117487,  0.1137996 , ..., -0.05892716,
        -0.4470452 , -0.10974538],
       [ 0.8692927 ,  0.31182288,  0.1137996 , ...,  1.45476435,
        -0.4470452 , -0.10974538],
       [-1.36836471,  0.9242922 ,  0.1137996 , ..., -0.05892716,
        -0.4470452 , -0.10974538],
       ...,
       [ 0.1234069 , -0.45376377,  0.1137996 , ..., -0.05892716,
         1.04673998, -0.10974538],
       [-0.62247891,  0.77117487,  0.1137996 , ..., -0.05892716,
         1.04673998, -0.10974538],
       [-1.36836471,  0.31182288,  0.1137996 , ..., -0.05892716,
        -0.4470452 , -0.10974538]])

In [23]:
y_predict = gbr.predict(feature_standard)
y_predict
df_train = pd.DataFrame(y_predict, columns=['price'])
df_train


,price
0,17374.359783
1,16452.153749
2,24365.876121
3,14649.738675
4,4781.646913
...,...
406,26984.329939
407,14053.073488
408,14080.847489
409,16067.001954


In [25]:
combined_df = pd.concat([test['Id'], df_train], axis=1)

combined_df


,Id,price
0,0,17374.359783
1,1,16452.153749
2,2,24365.876121
3,3,14649.738675
4,4,4781.646913
...,...,...
406,406,26984.329939
407,407,14053.073488
408,408,14080.847489
409,409,16067.001954


In [ ]:
combined_df.to_csv('Reha_car.csv', index=False)